This notebook demonstrates the process to extract data from RACECAR dataset and visulize them by input 'Translation' column sorted by 'Description'.

Firstly, we need to import nuScene module from python virtual environment

In [1]:
from nuscenes.nuscenes import NuScenes

The actual datasets needs to be imported from appropriate path.

In [2]:
nusc = NuScenes(version='v1.0-mini', dataroot='data/nuscenes/', verbose=True)

Loading NuScenes tables for version v1.0-mini...
1 category,
1 attribute,
1 visibility,
1 instance,
11 sensor,
78 calibrated_sensor,
369264 ego_pose,
22 log,
22 scene,
38108 sample,
1491862 sample_data,
1 sample_annotation,
2 map,
Done loading in 11.485 seconds.
Reverse indexing ...
Done reverse indexing in 4.0 seconds.


Once the dataset has been loaded, using nusc.list_scenes() to display all the scenes recorded during racing car event.

In [3]:
nusc.list_scenes()

SOLO-SLOW-70, Solo laps at speeds below 70 mph [70-01-01 04:32:25]    0s, IMS, #anns:0
SOLO-FAST-100-140, Solo laps between 100 and 140 mph [70-01-01 04:32:30]    0s, IMS, #anns:0
PYLON, Solo laps with pylon avoidance [70-01-01 04:32:30]    0s, IMS, #anns:0
SOLO-FAST-140, Solo laps above 140 mph [70-01-01 04:32:30]    0s, IMS, #anns:0
SOLO-FAST-100-140, Solo laps between 100 and 140 mph [70-01-01 04:32:30]    0s, IMS, #anns:0
SOLO-SLOW-70, Solo laps below 70 mph [70-01-01 04:33:16]    0s, LVMS, #anns:0
SOLO-SLOW-70, Solo laps below 70 mph [70-01-01 04:33:27]    0s, LVMS, #anns:0
SOLO-FAST-100-140, Solo laps between 100 and 140 mph [70-01-01 04:33:27]    0s, LVMS, #anns:0
SOLO-SLOW-70-100, Solo laps between 70 and 100 mph [70-01-01 04:33:28]    0s, LVMS, #anns:0
MULTI-SLOW-KAIST, Low speed multi-agent laps with KAIST [70-01-01 04:33:31]    0s, LVMS, #anns:0
MULTI-SLOW-MIT, Low speed multi-agent laps with MIT... [70-01-01 04:33:31]    0s, LVMS, #anns:0
MULTI-SLOW-TUM, Low speed multi-age

The scenes where this thesis project chose are 'Multi-agent fast', which are supposed to be 19, 21, and 22 data record from the list. However, it doesnt remain the consistency when it is using nusc.scene [].

Therefore, the scene of 'Multi-agent fast' were selected manually from the list, which are 7, 8, and 19, considering the indice starts at 0, it shoudl be 6, 7, and 18 in the code.

In [4]:
my_scene1=nusc.scene[18]
my_scene2=nusc.scene[6]
my_scene3=nusc.scene[7]



print(my_scene1)
print(my_scene2)
print(my_scene3)

{'description': 'High speed multi-agent laps with TUM', 'first_sample_token': '503ff877b528f907ee7ceb6f24a0dbd2', 'last_sample_token': 'e280475667087aa0695761d84cf4c38a', 'log_token': '634fc63aa088dc388b2e9a7bdfab5b5b', 'name': 'MULTI-FAST-TUM', 'nbr_samples': 1319, 'token': 'b02a9924e4cddd4bbb796e88238fe519'}
{'description': 'High speed multi-agent laps with Euroracing', 'first_sample_token': '89efc01ab67308eb9610d8761c320358', 'last_sample_token': 'c10a7232bb35244bf6b3a1b8e007b048', 'log_token': '0194ad49878c8290e661b26b1ec3b53b', 'name': 'MULTI-FAST-EURO', 'nbr_samples': 1469, 'token': 'd389494ffb3f31ad8bd534c50f702d9f'}
{'description': 'High speed multi-agent laps with Polimove', 'first_sample_token': '830dd14fc3ddd35d173d66a1031f6cce', 'last_sample_token': '6e32103b563f433b2db43ed35d5eedc4', 'log_token': 'f83d806b9db6984c9f5f4f6cadb1087f', 'name': 'MULTI-FAST-POLI', 'nbr_samples': 1599, 'token': 'fcbdd0c93960dcde30b965a78b6e73d6'}


The selected scenes are from 'Multi-agent fast', which have three data records from 'TUM', 'Euroracing', and 'Polimove',  those are also the strating point because we use 'first_sample_token'to generate sample iteraction, we could use 'nbr_samples' to verify the exract process, 'log_token' could be the key to find which vehicle is using to operate corresponding scence.

The next step is to find the sample based on select scene, the following code demonstrated the most difficult one to retreive.

The first thing is to define a sample_token from 'first_sample_token' in selected scene because that is the primary key to connect scene and its all samples. The predefined 'sensor1' and 'sensor2' are the primary key to retrive sample_data from sample, this demonstration utilize both because it is only solution to access all the sample_data.


Before the iterations, the log is retrived based on selected scene where 'log_token' as the primary key to identify the log respectively since every scene only has one corresponding log.

The important part is the iteration where the total times of loop should match 'nbr_samples' from scene to list complete samples.

In the iteration, sample, sample_data are using nusc.get() method to access corresponding data from scene, the important thing for scene[7] is that it must using both sensors to obtain the data, if 'LIDAR_FUSED' is not working, then it must use 'RADAR_FRONT', only setting one value for sensor is going to cause error for this process.

Finally, the 'pose' represents the 'ego_pose' where it stores all the location data, but at this point, it must be the ones from 'sample_data' because it follows the 'sample' from selected scenes. In a word, the listed 'ego_pose' must match log and scene that are predfined.

In [5]:
my_scene1=nusc.scene[7]
sample_token = my_scene1['first_sample_token']
sensor2='RADAR_FRONT'
sensor1='LIDAR_FUSED'
lg=nusc.get('log',my_scene1['log_token'])

print(lg)
print(my_scene1)

print('The following contents are samples')


for i in range(my_scene1['nbr_samples']):

    sample=nusc.get('sample', sample_token)
    

    try:
        data=nusc.get('sample_data',sample['data'][sensor1])
    except KeyError:
        
        data=nusc.get('sample_data',sample['data'][sensor2])
        
        
    pose=nusc.get('ego_pose',data['ego_pose_token'])
  
  
    print('------------',sample)

    print('------------',data)
    print('----------',i)
    print('------------',pose)
   

    sample_token=sample['next']

{'date_captured': '2022-01-07', 'location': 'LVMS', 'logfile': 'MULTI-FAST-POLI', 'token': 'f83d806b9db6984c9f5f4f6cadb1087f', 'vehicle': 'TUM', 'map_token': '7e3b5b651fe150647024ed0a72fe70fe'}
{'description': 'High speed multi-agent laps with Polimove', 'first_sample_token': '830dd14fc3ddd35d173d66a1031f6cce', 'last_sample_token': '6e32103b563f433b2db43ed35d5eedc4', 'log_token': 'f83d806b9db6984c9f5f4f6cadb1087f', 'name': 'MULTI-FAST-POLI', 'nbr_samples': 1599, 'token': 'fcbdd0c93960dcde30b965a78b6e73d6'}
The following contents are samples
------------ {'next': '0bc9b34400b98f51fa1397034b443b43', 'prev': '', 'scene_token': 'fcbdd0c93960dcde30b965a78b6e73d6', 'timestamp': 16415984005.002932, 'token': '830dd14fc3ddd35d173d66a1031f6cce', 'data': {'RADAR_FRONT': '69ad501e28a7b6b976796edc1c8c54ab', 'LIDAR_FUSED': 'ab2c8aebcdb232d00a56006935ba68a0'}, 'anns': []}
------------ {'calibrated_sensor_token': '3ddb7b64926c43c4d1ae107452ecb9ce', 'ego_pose_token': '60c1316311c901bfbc343378ebc1704d',

In the listing results, we could verify from i that it has 1598 values where matches the 'nbr_samples' from scene[7] because i starts the iteraction from 0.


This section contains the process to extract all the required dataset to a CSV file.

We need to utilize pandas module from python alias.

All the selected scene are defined as my_scene1, my_scene2, and my_scene3 in 'Multi-fast'

We use 'DataFrame'method from pands to define the colums that we need to extract, at this moment, the priamry attrbutes are Track, Vehicle, Description, Translation, Rotation, and Timestamp.
 
 
The sensors are predefined involved all the selected scenes from 'LIDAR_FUSED', 'RADAR_FRONT', and 'LIDAR_FRONT'.


The first iteration to go through each scene, it needs to define indice separately because selected ones are not sequential. We pass the 'first_sample_token' to variable to become starting point to generate sample.


The second iteration is limited by number of samples from selected scenes, there is a 'if' section because scene [7] must utilize both 'LIDAR_FUSED' and 'LIDAR_FRONT' to access and extract all the samples. The log here is based on scene we choose, where it has 'log_token' to point the log data that we can obtain Track and Vehicle.

From third to sixth iteration, it is using the sample pattern which 'entry_xx' represent the corresponding dataset based on selected scene as strating point because those are targeting data. The core of this process is to reversly access the data from'ego_pose' where it has translation and rotation information, which they rely on sample_data that it must utilize sensor to access from sample, and samples must match the current selected scene.


At the end of iteration, all the extact data passes to 'table_row', the print remain to verify the consistency between output from notebook and CSV file. The indicator 'i' servees for correct accumulation in each iteration. The sample_token must change into next one after extract every complete record.


Finally, we 'pd._to_csv' to export all the data this section has extracted and display them for verification.

In [6]:
#This cell is attemting for exporting dataset--Developing
import pandas as pd
my_scene1=nusc.scene[18]
my_scene2=nusc.scene[6]
my_scene3=nusc.scene[7]



df=pd.DataFrame(columns=['Track','Vehicle', 'Description','Translation','Rotation','Timestamp'])
    

sensor1='LIDAR_FUSED'
sensor2='RADAR_FRONT'
sensor3='LIDAR_FRONT'

 
    
            
for j in [6,7,18]:
    
    
    sample_token = nusc.scene[j]['first_sample_token']
 
    
    #print(nusc.scene[j])
    
  
    scenes=nusc.scene[j]
 
        
    

    for i in range(scenes['nbr_samples']):
    
    
    
        
        log=nusc.get('log',scenes['log_token'])
        samples=nusc.get('sample', sample_token)

        if j==18:
            sample_data=nusc.get('sample_data',samples['data'][sensor3])
        else: 
            try:    
                sample_data=nusc.get('sample_data',samples['data'][sensor1])
        
            except KeyError:
           
                sample_data=nusc.get('sample_data',samples['data'][sensor2])
        
       
       
        ego_pose=nusc.get('ego_pose',sample_data['ego_pose_token'])

    
        for logs in log:
    
            log_token=log.get('token')
            entry_scenes=[scene for scene in scenes if scenes['log_token']==log_token]
 
            for scene in entry_scenes:
        
                scene_token=scenes.get('token')
                entry_samples = [sample for sample in samples if samples['scene_token'] == scene_token]
        
        
   
                for sample in entry_samples:
                    track=log.get('location','')
                    vehicle=log.get('vehicle','')
                    description=scenes.get('description','')
                    timestamp=samples.get('timestamp','')
                    sample_token=samples.get('token')
            
                    entry_sampledata=[sampledata for sampledata in sample_data if sample_data['sample_token']==sample_token]
            
            
            
                    for sampledata in entry_sampledata:
                
                        ego_token=sample_data['ego_pose_token']
                        entry_ego=[ego for ego in ego_pose if ego_pose['token']==ego_token]
                        for ego in entry_ego:
                            translation=ego_pose.get('translation','')
                            rotation=ego_pose.get('rotation','')
                
            
           
   
            
        table_row = {'Track':track,'Vehicle': vehicle, 'Description': description,'Translation':translation,'Rotation':rotation,'Timestamp': timestamp}
        print(table_row)
        print(i)
        df1 = pd.DataFrame.from_records([table_row])
        sample_token=samples['next']
               
        df=pd.concat([df,df1],ignore_index=True)




df.to_csv('vechile_info.csv',index=False)

df
    

    

{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-244.80750189514848, 104.85640724830641, 2.0183713120128743], 'Rotation': [0.2525547609061316, 0.0, 0.0, 0.9675826025428769], 'Timestamp': 16415956005.12345}
0
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-244.9607810834281, 106.65502319518784, 2.018690583385265], 'Rotation': [-0.33470629322641354, 0.0, 0.0, 0.9423225017342174], 'Timestamp': 16415956010.12345}
1
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-247.12494631495903, 104.32739838520885, 2.0209707287875247], 'Rotation': [0.887020063530858, 0.0, 0.0, 0.46173088145987434], 'Timestamp': 16415956015.12345}
2
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-247.6832689784443, 105.01430466228736, 2.0250903892534], 'Rotat

/tmp/ipykernel_193644/3495734761.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df=pd.concat([df,df1],ignore_index=True)


{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-245.47743457074296, 104.21119462485991, 2.016288232990135], 'Rotation': [0.5221907844180572, 0.0, 0.0, 0.8528286959693924], 'Timestamp': 16415956115.12345}
22
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-244.6195371017145, 105.87510337404676, 2.017746558454199], 'Rotation': [-0.0731517750797938, 0.0, 0.0, 0.997320819898329], 'Timestamp': 16415956120.12345}
23
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-245.55203524675213, 104.17505979609251, 2.0191055227545363], 'Rotation': [0.5438217994675001, 0.0, 0.0, 0.8392007211769602], 'Timestamp': 16415956125.12345}
24
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-245.14700215925816, 104.4347200497629, 2.0194083986661937], 'Ro

{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-349.507384614451, -21.46931162637246, 3.1150382137004176], 'Rotation': [0.27897122584037315, 0.0, 0.0, 0.960299461185478], 'Timestamp': 16415956285.12345}
56
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-357.3342872537296, -34.0329251733078, 3.2279192712420475], 'Rotation': [0.2729136998830844, 0.0, 0.0, 0.9620385191956327], 'Timestamp': 16415956290.12345}
57
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-362.6152601680609, -42.51626662726058, 3.303167617048363], 'Rotation': [0.2753231109875236, 0.0, 0.0, 0.9613517486103367], 'Timestamp': 16415956295.12345}
58
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-370.39482888593454, -55.39374845023312, 3.39780745621508], 'Rotati

{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-180.3486971873474, -386.68800699016515, 4.08171612593712], 'Rotation': [-0.6953206168332551, 0.0, 0.0, 0.7186996868001416], 'Timestamp': 16415956480.12345}
95
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-170.10644950115602, -386.75107106923633, 4.126061404542938], 'Rotation': [0.7128449817245268, 0.0, 0.0, -0.7013216323700269], 'Timestamp': 16415956485.12345}
96
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-154.7977862546622, -385.81254419233784, 4.19005058340845], 'Rotation': [0.7410113428091528, 0.0, 0.0, -0.6714925091377985], 'Timestamp': 16415956490.12345}
97
{'Track': 'LVMS', 'Vehicle': 'TUM', 'Description': 'High speed multi-agent laps with Euroracing', 'Translation': [-144.6740994824643, -384.45801953248923, 4.2392050668139944], 

...

{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [260.5822786006328, -105.6822533106554, 4.663226059503863], 'Rotation': [0.9445417310847145, 0.0, 0.0, 0.32839141011830814], 'Timestamp': 16415853565.03897}
1132
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [286.2216307491643, -85.34738843454218, 4.69951238966388], 'Rotation': [0.9435137337789777, 0.0, 0.0, 0.33133341843293174], 'Timestamp': 16415853570.03897}
1133
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [301.11300104051736, -73.16168020516832, 4.694777668612097], 'Rotation': [0.9393712982190319, 0.0, 0.0, 0.34290168282219163], 'Timestamp': 16415853575.03897}
1134
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [327.91813321703546, -48.98660114788444, 4.607341665751484], '

{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-353.6766201130526, 17.262888706098224, 3.963011339542607], 'Rotation': [-0.5474031155724651, 0.0, 0.0, 0.8368690632718827], 'Timestamp': 16415853785.03897}
1176
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-369.3255034449382, -17.628439295450715, 4.231622897151227], 'Rotation': [-0.541595684712461, 0.0, 0.0, 0.84063911061813], 'Timestamp': 16415853790.03897}
1177
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-379.6531188150177, -40.73722938390547, 4.420522154607774], 'Rotation': [-0.5482759820254275, 0.0, 0.0, 0.8362974635463467], 'Timestamp': 16415853795.03897}
1178
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-393.38127538100633, -75.27962668415034, 4.538643474392302],

{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [352.8290198513701, 293.2488266663713, 3.609090065577016], 'Rotation': [0.39613560951603316, 0.0, 0.0, 0.9181920163415498], 'Timestamp': 16415854060.03897}
1231
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [336.5914260178427, 308.9866015099654, 3.493359157981301], 'Rotation': [0.35232430536610876, 0.0, 0.0, 0.9358779748708103], 'Timestamp': 16415854065.03897}
1232
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [309.3736987127785, 329.40219912767003, 3.2982250305473215], 'Rotation': [0.2787252438809026, 0.0, 0.0, 0.9603708858683355], 'Timestamp': 16415854070.03897}
1233
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [289.5999908825185, 340.67859029794323, 3.2087773140851965], 'Ro

{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-135.70844718152182, -393.2996662789102, 5.583315961289529], 'Rotation': [0.9951924841733911, 0.0, 0.0, 0.09793834511974578], 'Timestamp': 16415854310.03897}
1281
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-110.62260660944644, -386.86501379972844, 5.662658793847314], 'Rotation': [0.9881920333781361, 0.0, 0.0, 0.15322044631179188], 'Timestamp': 16415854315.03897}
1282
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-74.64862549758553, -372.15656810953544, 5.461288436646321], 'Rotation': [0.9735330522684205, 0.0, 0.0, 0.228546266959084], 'Timestamp': 16415854320.03897}
1283
{'Track': 'LVMS', 'Vehicle': 'EURORACING', 'Description': 'High speed multi-agent laps with TUM', 'Translation': [-51.96058241844704, -359.56963951548664, 5.3453395387972

,Track,Vehicle,Description,Translation,Rotation,Timestamp
0,LVMS,TUM,High speed multi-agent laps with Euroracing,"[-244.80750189514848, 104.85640724830641, 2.01...","[0.2525547609061316, 0.0, 0.0, 0.9675826025428...",1.641596e+10
1,LVMS,TUM,High speed multi-agent laps with Euroracing,"[-244.9607810834281, 106.65502319518784, 2.018...","[-0.33470629322641354, 0.0, 0.0, 0.94232250173...",1.641596e+10
2,LVMS,TUM,High speed multi-agent laps with Euroracing,"[-247.12494631495903, 104.32739838520885, 2.02...","[0.887020063530858, 0.0, 0.0, 0.46173088145987...",1.641596e+10
3,LVMS,TUM,High speed multi-agent laps with Euroracing,"[-247.6832689784443, 105.01430466228736, 2.025...","[0.9798854525184691, 0.0, 0.0, 0.1995607675438...",1.641596e+10
4,LVMS,TUM,High speed multi-agent laps with Euroracing,"[-244.61341998435506, 105.85857302127593, 2.02...","[-0.06771597065140982, 0.0, 0.0, 0.99770463931...",1.641596e+10
...,...,...,...,...,...,...
4382,LVMS,EURORACING,High speed multi-agent laps with TUM,"[148.89305280842757, -179.6919014898091, 3.999...","[0.9608727807482902, 0.0, 0.0, 0.2769900706109...",1.641585e+10
4383,LVMS,EURORACING,High speed multi-agent laps with TUM,"[149.93417767425663, -177.26191844774348, 3.99...","[0.7739725484092828, 0.0, 0.0, -0.633219151880...",1.641585e+10
4384,LVMS,EURORACING,High speed multi-agent laps with TUM,"[148.69984005591675, -178.43994148137455, 4.00...","[0.9923303709532226, 0.0, 0.0, -0.123614056174...",1.641585e+10
4385,LVMS,EURORACING,High speed multi-agent laps with TUM,"[150.41171546343048, -177.2378526873551, 3.999...","[-0.6721141295736687, 0.0, 0.0, 0.740447565211...",1.641585e+10


The output verifed that the listing results are consistent from CSV file dataset, at this point, this function of this notebook has been finalzied as extraction and export.